# This is my third assignment with website scrapping

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
import requests 
from bs4 import BeautifulSoup
import csv
import pandas as pd
import numpy as np

In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 

In [4]:
soup = BeautifulSoup(r.content, 'html5lib') 

In [5]:
quotes=[]

table_data = soup.find('table', attrs = {'class':'wikitable sortable'}) 

row_marker = 0
quote={}
for row in table_data.find_all('tr'):

     column_marker = 0
     columns = row.find_all('td')
     
     quotes.append(quote) 
     quote={}
     
     for column in columns:
         
         if column_marker == 0:
            quote['Postcode'] = column.get_text()
            
        
         if column_marker == 1:
            quote['Borough'] = column.get_text()
            
        
         if column_marker == 2:
            quote['Neighbourhood'] = column.get_text()
            
            
            
         
         column_marker += 1
         
     
       


df2 = pd.DataFrame(quotes)

df2.drop([1], axis = 0, inplace = True)
newdf = df2[df2['Borough'] != 'Not assigned']
newdf['Neighbourhood'] = newdf['Neighbourhood'].map(lambda x: str(x)[:-1])
newdf = newdf.reset_index(drop=True)


newdf['Neighbourhood'] = np.where(newdf['Neighbourhood'] == 'Not assigned', newdf['Borough'], newdf['Neighbourhood'])


newdf = newdf['Neighbourhood'].groupby([newdf.Postcode, newdf.Borough]).apply(', '.join).reset_index()


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
url1='http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
coord_df = pd.read_csv(url1)

In [7]:
merge_df = pd.merge(newdf, coord_df, left_on='Postcode', right_on='Postal Code', how='left').drop('Postal Code', axis=1)

In [8]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.653963, -79.387207.


In [9]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merge_df['Latitude'], merge_df['Longitude'], merge_df['Borough'], merge_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto